<div align='center'><font size="5" color='#353B47'>CARS PRICING</font></div>
<div align='center'><font size="4" color="#353B47">PART2</font></div>
<br>
<div align="center"><img alt="Super Sports Cars Wallpaper  HD Car Wallpapers" height="auto" src="http://www.hdcarwallpapers.com/download/super_sports_cars-1366x768.jpg" title="Super Sports Cars Wallpaper  HD Car Wallpapers" width="65%"></div>
<br>
<div align='center'><font size = '4' color='#353B47'>EDA and preprocessing of the data: <a href = 'https://www.kaggle.com/bryanb/simple-and-quick-eda'>CARS PRICING - PART 1</a></font></div>
<hr>

## <div id="summary">Table of contents</div>

**<font size="2"><a href="#chap1">Introduction</a></font>**
<br>
**<br><font size="2"><a href="#chap2">2. Data description</a></font>**
**<br><font size="2"><a href="#chap3">3. Model</a></font>**
**<br><font size="2"><a href="#chap4">4. SHapley Additive exPlanations</a></font>**
**<br><font size="2"><a href="#chap5">5. Predictions</a></font>**
<br>
**<br><font size="2"><a href="#chap6">Conclusion</a></font>**

> ### <div id="chap1">Introduction</div>

<div align='justify'><font size=3>The data come from the Kaggle dataset Car Features and MSRP of Kaggle. It describes almost 12 000 car models sold in the USA between 1990 and 2018 with the market price (new or used) and some features. This study aims at performing some data manipulations and define a statistical model to predict the price of a car.</font></div>

### <div id="chap2"> 2. Data description</div>

In [ ]:
# Make                Car Make
# Model               Car Model
# Year                Car Year (Marketing)
# Engine Fuel Type    Engine Fuel Type
# Engine HP           Engine Horse Power (HP)
# Engine Cylinders    Engine Cylinders
# Transmission Type   Transmission Type
# Driven_Wheels       Driven Wheels
# Number of Doors     Number of Doors
# Market Category     Market Category
# Vehicle Size        Size of Vehicle
# Vehicle Style       Type of Vehicle
# highway MPG         Highway MPG
# city mpg            City MPG
# Popularity          Popularity (Twitter)
# MSRP                Manufacturer Suggested Retail Price

**<font color='blue' size='3'>2.1 Explanation of MPG</font>** 

<div align='justify'><font size=3>A measure of how far a car can travel if you put just one gallon of petrol or diesel in its tank.</font></div>

**<font color='blue' size='3'>2.2 Explanation on MSRP</font>**

<div align='justify'><font size=3>MSRP is the manufacturer's suggested retail price, a price recommended for the sale of an item in all retail outlets. A vendor can require retailers to sell its products at the MSRP and refuse to sell its products to a discounter if they price products below it.</font></div>

> - Prices retained
> 
>     ◦ for vehicles sold in 2017 (1 668 vehicles), new price suggested by the manufacturer
>
>     ◦ for used vehicles (10 246 vehicles), edmunds.com True Market Value
>     
This difference in method could lead to a discontinuity. However we see on different examples there isn't any significant gap.
> 
> - All prices are recorded at the same time and can be compared. There is no need for inflation ajustment.
> 
> - Used car prices are floored on edmunds.com. Oldest or cheapest vehicles have a minimum price of 2000 USD (1 036 vehicles).

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

### <div id="chap3"> 3. Setup</div>

#### <font color='blue' size='3'>3.1 Librairies</font>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection, ensemble
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

import time
import shap

#### <font color='blue' size='3'>3.2 Import datas</font>

In [ ]:
df_cleaned = pd.read_csv("../input/cars_cleaned.csv").drop('Unnamed: 0', axis=1)
df_cleaned.head(5)

In [ ]:
# Check missing values
len(df_cleaned) - df_cleaned.count()

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

## <div id="chap4"> 4. Model</div>

#### <font color='blue' size='3'>4.1 Encoding categorical features</font>

In [ ]:
d = defaultdict(LabelEncoder)
columns_to_encode = [
    "Make",
    "Model",
    "Engine Fuel Type",
    "Transmission Type",
    "Driven_Wheels",
    "Market Category",
    "Transmission Type",
    "Vehicle Size",
    "Vehicle Style"]

df_cleaned.loc[:,columns_to_encode] = df_cleaned.loc[:,columns_to_encode].apply(lambda x: d[x.name].fit_transform(x.fillna('0')))
df_cleaned.info()

#### <font color='blue' size='3'>4.2 Strategy selection</font>

In [ ]:
# GET DUMMIES, FEATURES SELECTION, PROPERLY CLEANED AND INPUTED DATAS

df_cleaned_dummies = df_cleaned.copy()
df_cleaned_dummies = pd.get_dummies(df_cleaned_dummies)

X = df_cleaned_dummies.drop(['MSRP'], axis=1)
y = df_cleaned_dummies['MSRP']

#### <font color='blue' size='3'>4.3 Split the data</font>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4, random_state = 42)

# Need index of car prices to predict, whatever the strategy is
indexes_cars_predicted = X_test.index

#### <font color='blue' size='3'>4.4 Feature selection</font>

##### <font color='orange' size='2'>How to chose NUM_OF_FEATURES to select ?</font>

In [ ]:
def rmsle(predicted, real):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

NUM_OF_FEATURES = X.shape[1]

train_X_train, train_X_test, train_y_train, train_y_test = train_test_split(X_train, y_train, test_size=0.2)

model = ensemble.RandomForestRegressor(n_jobs=-1, n_estimators = 100, random_state=42)
model.fit(train_X_train, train_y_train)

# Graphs section
fig = plt.figure(figsize=(15,5))
ax1 = plt.subplot(111)
plt.plot(np.cumsum(model.feature_importances_))
plt.axhline(0.85,color= 'r')

NUM_OF_FEATURES = 200

col = pd.DataFrame({'importance': model.feature_importances_, 'feature': X_train.columns}).sort_values(
    by=['importance'], ascending=[False])[:NUM_OF_FEATURES]['feature'].values

X_train = X_train[col]
X_test = X_test[col]

#### <font color='blue' size='3'>4.5 LGBM vs XGBOOST</font>

In [ ]:
# Define evaluation method for a given model. we use k-fold cross validation on the training set. 
# The loss function is root mean square logarithm error between target and prediction
# Note: train and y_train are feeded as global variables

NUM_FOLDS = 5
def rmsle_cv(model,strategy):
        kf = KFold(NUM_FOLDS, shuffle=True, random_state=42).get_n_splits(X_train.values)
        rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
        return(rmse)

# Ensemble method: model averaging
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # We define clones of the original models to fit the data in
    # the reason of clone is avoiding affect the original base models
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]  
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
        return self
    
    # Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([ model.predict(X) for model in self.models_ ])
        return np.mean(predictions, axis=1)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.055, 
                             gamma=1.5, 
                             learning_rate=0.02, 
                             max_depth=12, 
                             n_estimators=1000,
                             subsample=0.7, 
                             objective='reg:linear',
                             booster='gbtree',
                             reg_alpha=0.0, 
                             eval_metric = 'rmse', 
                             silent=1, 
                             random_state =7,
                            )

model_lgb = lgb.LGBMRegressor(objective='regression',
                              num_leaves=144,
                              learning_rate=0.005, 
                              n_estimators=1000, 
                              max_depth=12,
                              metric='rmse',
                              is_training_metric=True,
                              max_bin = 55, 
                              bagging_fraction = 0.8,
                              verbose=-1,
                              bagging_freq = 5, 
                              feature_fraction = 0.9)

l_m = []
l_std = []
time_d = {}

print("GET DUMMIES, FEATURES SELECTION, KEEP MISSING VALUES\n")

start = time.time()
score_1 = rmsle_cv(model_xgb,1)
end = time.time()

l_m.append(score_1.mean())
l_std.append(score_1.std())
time_d['XGB']= end - start
print("    Xgboost score : {:.4f} ({:.4f})".format(score_1.mean(), score_1.std()))

start = time.time()
score_1 = rmsle_cv(model_lgb,1)
end = time.time()

l_m.append(score_1.mean())
l_std.append(score_1.std())
time_d['LGB']= end - start
print("    LGBM score    : {:.4f} ({:.4f})" .format(score_1.mean(), score_1.std()))

averaged_models_1 = AveragingModels(models = (model_xgb, model_lgb))

score_1 = rmsle_cv(averaged_models_1,1)

l_m.append(score_1.mean())
l_std.append(score_1.std())
print("    Averaged score: {:.4f} ({:.4f})\n" .format(score_1.mean(), score_1.std()))

print('\nLGB faster than XGB ?',time_d['LGB']>time_d['XGB'])

LGBM is a great solution if your PC runs slowly, hence the word 'light'. It trains faster than XGBOOST. However I don't manipulate a significant amount of datas and given the way each algorithm is parametrized, the computation depends. On average LGBM is performs faster but with less accuracy than XGB.

#### <font color='blue' size='3'>4.6 The best model</font>

In [ ]:
model_lgb.fit(X_train.values, y_train)
pred = model_lgb.predict(X_test.values)

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

## <div id="chap5"> 5. SHapley Additive exPlanations</div>

#### <font color='blue' size='3'>5.1 What is a SHAP value</font>

SHAP is a unified approach to explain the output of any machine learning model. SHAP connects game theory with local explanations, representing the only possible consistent and locally accurate additive feature attribution method based on expectations (More details : http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions).

#### <font color='blue' size='3'>5.2 Visualization</font>

In [ ]:
# load JS visualization code to notebook
shap.initjs()

In [ ]:
# Using a random sample of the dataframe for better time computation
X_sampled = X_train.sample(100, random_state=10)

In [ ]:
# explain the model's predictions using SHAP values
# (same syntax works for LightGBM, CatBoost, and scikit-learn models)
explainer = shap.TreeExplainer(model_lgb)
shap_values = explainer.shap_values(X_sampled)

In [ ]:
# visualize the first prediction's explanation
shap.force_plot(explainer.expected_value, shap_values[0,:], X_sampled.iloc[0,:])

The above explanation shows features each contributing to push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue (these force plots are introduced in our Nature BME paper).

In [ ]:
# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values, X_train)

To understand how a single feature effects the output of the model we can plot the SHAP value of that feature vs. the value of the feature for all the examples in a dataset. SHAP values represent a feature's responsibility for a change in the model output. Vertical dispersion represents interaction effects with other features. To help reveal these interactions dependence_plot automatically selects another feature for coloring.

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X_sampled)

We can also just take the mean absolute value of the SHAP values for each feature to get a standard bar plot (produces stacked bars for multi-class outputs):

In [ ]:
shap.summary_plot(shap_values, X_sampled, plot_type="bar")

Given that representative sample, when someone wants to buy a car, the criterias will be the city mpg, the car age and the engine HP of the car.

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

## <div id="chap6"> 6. Predictions</div>

In [ ]:
pred = np.array(pred)
original = np.array(df_cleaned_dummies.loc[indexes_cars_predicted,'MSRP'])

df_cleaned_dummies.loc[indexes_cars_predicted,'MSRP']

def average_gap(l1,l2):
    resu=0
    for i in range(len(l1)):
        resu += np.abs(l1[i]-l2[i])
    resu = resu/len(l1)
    return(resu)

print("Over",len(pred),"cars, the average gap between the predicted price and the real price is",round(average_gap(pred,original),0),"$")

plt.figure(figsize=(15,7))
sns.distplot(pred, color="blue", label="Distrib Predictions", hist = False)
sns.distplot(original, color="red", label="Distrib Original", hist = False)
plt.title("Distribution of pred and original MSRP")
plt.legend()

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

## <div id="chap7">Conclusion</div>

<div align='justify'><font size=3>To improve the model, some features could be added such as the weight of the car, the nationality of the car manufacturer, options sold with the car and many others.</font></div>

## References

* **Understanding of LGBM, XGB** : https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db

* **Interpretation of SHAP values** : https://github.com/slundberg/shap 

<hr>
<br>
<div align='justify'><font color="#353B47" size="4">Thank you for taking the time to read this notebook. I hope that I was able to answer your questions or your curiosity and that it was quite understandable. <u>any constructive comments are welcome</u>. They help me progress and motivate me to share better quality content. I am above all a passionate person who tries to advance my knowledge but also that of others. If you liked it, feel free to <u>upvote and share my work.</u> </font></div>
<br>
<div align='center'><font color="#353B47" size="3">Thank you and may passion guide you.</font></div>